In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

########## CNN CODE ############

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 48, 48)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 24, 24)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 24, 24)
        )
        
        self.conv2 = nn.Sequential(         # input shape (16, 24, 24)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 24, 24)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 12, 12)
        )
        
        self.conv3 = nn.Sequential(         # input shape (32, 12, 12)
            nn.Conv2d(32, 64, 3, 1, 1),     # output shape (64, 12, 12)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (64, 6, 6)
        )
        self.out = nn.Linear(64 * 6 * 6, 5)   # fully connected layer, output 10 classes
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 14 * 14)
        output = F.softmax(self.out(x))
        return output


cnn = CNN()
print (cnn)

In [ ]:
# SGD needs more (like x100 more!) learning rate.
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.0003)   # optimize all cnn parameters
# TODO: Maybe add weight_decay parameter to the optimier?

loss_func = nn.CrossEntropyLoss() 

def make_batch(train, labels, batch_size=10, gpu=False):
    start = 0
    stop = start + batch_size
    while start < train.shape[0]:
        if gpu:
            yield Variable(torch.FloatTensor(train[start:stop]), requires_grad=True).cuda(), Variable(torch.LongTensor(labels[start:stop])).cuda()
        else:
            yield Variable(torch.FloatTensor(train[start:stop]), requires_grad=True), Variable(torch.LongTensor(labels[start:stop]))
        start = stop
        stop = start + batch_size


def train_it(X_train,y_train):

	for epoch in range(1000):
		for step, (b_x, b_y) in enumerate(make_batch(X_train, y_train, 1)):   # gives batch data, normalize x when iterate train_loader
			# print step,
			output = cnn(b_x) # cnn output
			#print(output.shape)
			# print output.size(), output.sum(dim=0)
			#print("b_y.shape",b_y.shape)
			loss = loss_func(output, b_y)   # cross entropy loss
			optimizer.zero_grad()           # clear gradients for this training step
			loss.backward()                 # backpropagation, compute gradients

			# for params in cnn.parameters():
			#     print params.grad.cpu().data.sum() # Y U no train!!!
			optimizer.step()                # apply gradients

			if epoch % 100 == 0:
			    #test_output = cnn(torch.FloatTensor(X_test))
			    #outs = test_output.cpu().data.numpy().argmax(axis=1)
			    #acc = (outs == y_test).sum()*100.0 / test_output.shape[0]
			    # pred_y = torch.max(test_output, 1)[1].data.squeeze().numpy()
			    # accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
			    print('Epoch: ', epoch, '| Step: ', step)

def predict_it(X_test):
	test_output = cnn(torch.FloatTensor(X_test))
	print("test_output",test_output)
	outs = test_output.cpu().data.numpy().argmax(axis=1)
	#print("outs",outs)
	return outs


In [ ]:
dataset_path = 'D:/data/'

face_data = []
labels = []
class_id = 0

# Dataset prepration
for fx in os.listdir(dataset_path):
	if fx.endswith('.npy'):
		data_item = np.load(dataset_path + fx)
		print("data_item.shape   :",data_item.shape)
		data_item = cv2.resize(data_item, (48, 48))
		print("data_item.shape  after resize  :",data_item.shape)
		face_data.append(data_item)

		#target = class_id * np.ones((data_item.shape[0],))
		labels.append(class_id)
		class_id += 1


face_dataset = np.concatenate(face_data, axis=0)
#face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))
#print ("face_labels.shape",face_labels.shape)
print ("face_dataset.shape",face_dataset.shape)

X_train = face_dataset.reshape((-1, 1, 48, 48)) / 255.0

y_train = np.asarray(labels)

print("X_train.shape",X_train.shape)
print("y_train.shape",y_train.shape)

In [ ]:
#Training DATA

train_it(X_train,y_train)

In [ ]:
# Initialize camera
cap = cv2.VideoCapture(0)

# Load the haar cascade for frontal face
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')
  

print("labels: ",labels)
#trainset = np.concatenate((face_dataset, face_labels), axis=1)
#print (trainset.shape)

names = {
	0: 'ankit',        
	1: 'Dhoni',
	2: 'Himanshu',
	3: 'Modi',
	4: 'Vimal'
}

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
	ret, frame = cap.read()
	if ret == False:
		continue
	# Convert frame to grayscale
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	# Detect multi faces in the image
	faces = face_cascade.detectMultiScale(gray, 1.3, 5)

	for face in faces:
		x, y, w, h = face

		# Get the face ROI
		offset = 7
		face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        
# 		plt.figure(0)
# 		plt.title("orignal")
# 		plt.imshow(face_section)
# 		plt.show() 
		face_section=cv2.cvtColor(face_section,cv2.COLOR_BGR2GRAY)
		plt.figure(0)
		plt.title("Gray")
		plt.imshow(face_section)
		plt.show() 
		face_section = cv2.resize(face_section, (48, 48))
		print("face_section",face_section.shape)
        
		plt.figure(0)
		plt.title("Resized")
		plt.imshow(face_section)
		plt.show() 

       
		test = face_section.reshape((-1, 1, 48, 48)) / 255.0
		out=predict_it(test)
		#out = knn(trainset, face_section.flatten())
        
		#print("out",out[0])
		#counts = np.bincount(out)
		#print(np.argmax(counts))
		# Draw rectangle in the original image
		cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
		cv2.putText(frame, names[int(out[0])],(x,y-10), font, 1,(255,0,0),2,cv2.LINE_AA)


	cv2.imshow("Faces", frame)

	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cv2.destroyAllWindows()